In [12]:
from IPython.display import HTML

js_code = '''
<script>
class JobDescription {
    constructor(title, required_skills, required_experience_years, required_qualification) {
        this.title = title;
        this.required_skills = required_skills;
        this.required_experience_years = required_experience_years;
        this.required_qualification = required_qualification;
    }
}

class Resume {
    constructor(candidate_name, skills, experience_years, qualification) {
        this.candidate_name = candidate_name;
        this.skills = skills;
        this.experience_years = experience_years;
        this.qualification = qualification;
    }
}

function matchSkills(resume, jobDescription) {
    /**
     * Compares skills from a resume with required skills from a job description.
     *
     * @param {Resume} resume - The candidate's resume object.
     * @param {JobDescription} jobDescription - The job description object.
     * @returns {number} The skill match score between 0 and 1.
     */
    const resumeSkills = new Set(resume.skills.map(skill => skill.toLowerCase()));
    const requiredSkills = new Set(jobDescription.required_skills.map(skill => skill.toLowerCase()));

    const commonSkills = [...resumeSkills].filter(skill => requiredSkills.has(skill)).length;

    if (requiredSkills.size === 0) {
        // If no skills are required, consider it a perfect match if the resume has any skills, or 1.0
        return 1.0;
    }

    const score = commonSkills / requiredSkills.size;
    return score;
}

function matchExperience(resume, jobDescription) {
    /**
     * Compares the candidate's experience with the required experience for the job.
     *
     * @param {Resume} resume - The candidate's resume object.
     * @param {JobDescription} jobDescription - The job description object.
     * @returns {number} The experience match score between 0 and 1.
     */
    const requiredExp = jobDescription.required_experience_years;
    const candidateExp = resume.experience_years;

    if (requiredExp === 0) {
        return 1.0;
    } else if (candidateExp >= requiredExp) {
        return 1.0;
    } else {
        return candidateExp / requiredExp;
    }
}

function matchQualification(resume, jobDescription) {
    /**
     * Compares the candidate's highest qualification with the required qualification for the job.
     *
     * @param {Resume} resume - The candidate's resume object.
     * @param {JobDescription} jobDescription - The job description object.
     * @returns {number} The qualification match score between 0 and 1.
     */
    const qualificationLevels = {
        "none": 0,
        "high school": 1,
        "associate's degree": 2,
        "bachelor's degree": 3,
        "master's degree": 4,
        "phd": 5
    };

    const candidateQual = resume.qualification ? resume.qualification.toLowerCase() : "none";
    const requiredQual = jobDescription.required_qualification ? jobDescription.required_qualification.toLowerCase() : "none";

    const candidateLevel = qualificationLevels[candidateQual] || 0;
    const requiredLevel = qualificationLevels[requiredQual] || 0;

    // If required qualification is not in mapping or very basic and candidate has any qual, return 1.0
    if ((requiredLevel === 0 || ["none", "high school"].includes(requiredQual)) && candidateLevel > 0) {
        return 1.0;
    }

    if (requiredLevel === 0 && candidateLevel === 0) {
        return 1.0; // Both have no specific qualification, consider it a match
    }

    if (candidateLevel >= requiredLevel) {
        return 1.0;
    } else if (requiredLevel > 0) { // Ensure no division by zero
        return candidateLevel / requiredLevel;
    } else {
        return 0.0; // Should not be reached if conditions above are correct, but as a fallback
    }
}

function calculateOverallScore(resume, jobDescription) {
    /**
     * Calculates the overall match score between a resume and a job description.
     *
     * @param {Resume} resume - The candidate's resume object.
     * @param {JobDescription} jobDescription - The job description object.
     * @returns {number} The overall match score between 0 and 1.
     */
    // Define weights for each matching category
    // These weights can be adjusted based on importance
    let skillWeight = 0.5;
    let experienceWeight = 0.3;
    let qualificationWeight = 0.2;

    // Ensure weights sum up to 1 for normalization
    const totalWeight = skillWeight + experienceWeight + qualificationWeight;
    if (totalWeight !== 1.0) {
        skillWeight /= totalWeight;
        experienceWeight /= totalWeight;
        qualificationWeight /= totalWeight;
    }

    // Get individual match scores
    const skillScore = matchSkills(resume, jobDescription);
    const experienceScore = matchExperience(resume, jobDescription);
    const qualificationScore = matchQualification(resume, jobDescription);

    // Calculate the weighted overall score
    const overallScore = (
        skillScore * skillWeight +
        experienceScore * experienceWeight +
        qualificationScore * qualificationWeight
    );

    return overallScore;
}

// Example Usage (You can uncomment and run this section to test)
// Define a job description
const job1 = new JobDescription(
    "Software Engineer",
    ["Python", "Java", "SQL", "Django"],
    3,
    "bachelor's degree"
);

// Define some resumes
const resume1 = new Resume(
    "Alice Smith",
    ["Python", "SQL", "C++", "Django"],
    4,
    "master's degree"
);

const resume2 = new Resume(
    "Bob Johnson",
    ["Java", "C#"],
    2,
    "bachelor's degree"
);

const resume3 = new Resume(
    "Charlie Brown",
    ["Python", "SQL"],
    1,
    "high school"
);

// Calculate and print scores
console.log(`\nMatching ${resume1.candidate_name} to ${job1.title}:`);
console.log(`Skill Score: ${matchSkills(resume1, job1).toFixed(2)}`);
console.log(`Experience Score: ${matchExperience(resume1, job1).toFixed(2)}`);
console.log(`Qualification Score: ${matchQualification(resume1, job1).toFixed(2)}`);
const overallScore1 = calculateOverallScore(resume1, job1);
console.log(`Overall Match Score: ${overallScore1.toFixed(2)} (${(overallScore1 * 100).toFixed(0)}%)\n`);

console.log(`Matching ${resume2.candidate_name} to ${job1.title}:`);
console.log(`Skill Score: ${matchSkills(resume2, job1).toFixed(2)}`);
console.log(`Experience Score: ${matchExperience(resume2, job1).toFixed(2)}`);
console.log(`Qualification Score: ${matchQualification(resume2, job1).toFixed(2)}`);
const overallScore2 = calculateOverallScore(resume2, job1);
console.log(`Overall Match Score: ${overallScore2.toFixed(2)} (${(overallScore2 * 100).toFixed(0)}%)\n`);

console.log(`Matching ${resume3.candidate_name} to ${job1.title}:`);
console.log(`Skill Score: ${matchSkills(resume3, job1).toFixed(2)}`);
console.log(`Experience Score: ${matchExperience(resume3, job1).toFixed(2)}`);
console.log(`Qualification Score: ${matchQualification(resume3, job1).toFixed(2)}`);
const overallScore3 = calculateOverallScore(resume3, job1);
console.log(`Overall Match Score: ${overallScore3.toFixed(2)} (${(overallScore3 * 100).toFixed(0)}%)\n`);
</script>
'''

display(HTML(js_code))